In [1]:
# ===========================================
# Earth Engine Initialization and Authentication
# ===========================================

import ee

# Method 1: Interactive authentication (for local development or Jupyter Notebooks)
# This method will open a browser window to log in with your Google account.
# Use this when running locally for the first time or if no credentials are stored.
ee.Authenticate(quiet=True)

# Initialize Earth Engine (automatically uses default credentials if already authenticated)
ee.Initialize()

# Test the initialization
print(ee.String('GEE initialized').getInfo())

GEE initialized


In [2]:
# Method 2: Service account authentication (for automated pipelines or server deployment)
# Note: The service account JSON key file must be located in the same directory as this script.

credentials = ee.ServiceAccountCredentials(
     'changsome.1@gmail.com',  # Service account email
     'global-flood-mapping-db762f1c1e40.json'  # Path to service account key file
 )
ee.Initialize(credentials)

# Test the initialization
print(ee.String('GEE initialized').getInfo())

GEE initialized


In [2]:
# ===========================================
# Interactive Map Initialization
# ===========================================

# Import geemap, interactive mapping
import geemap

# Initialize an interactive map using geemap
m = geemap.Map()

# Note: If this is the last line of a cell, you can omit 'display()' and just write 'm'
display(m)

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [3]:
# ===========================================
# SAR Image Collection and Multi-Temporal Mean Visualization
# ===========================================

# Define a function to mask out the low-value edges of the SAR image
def mask_edge(image):
    """
    Masks the edge areas of a Sentinel-1 SAR image where backscatter is very low.
    
    Args:
        image (ee.Image): Input SAR image (VV polarization)
    
    Returns:
        ee.Image: Image with low-value edges masked out
    """
    edge = image.lt(-30.0)  # Identify areas with backscatter < -30 dB (likely noisy edges)
    masked_image = image.mask().And(edge.Not())  # Keep valid areas only
    return image.updateMask(masked_image)


# Create an ImageCollection of Sentinel-1 SAR images with VV polarization and IW mode
img_vv = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))  # VV polarization only
    .filter(ee.Filter.eq('instrumentMode', 'IW'))  # Interferometric Wide (IW) mode
    .select('VV')  # Select VV band
    .map(mask_edge)  # Apply edge masking function
)

# Split the collection into descending and ascending orbit tracks
desc = img_vv.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
asc = img_vv.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))

# Define seasonal date filters
spring = ee.Filter.date('2015-03-01', '2015-04-20')
late_spring = ee.Filter.date('2015-04-21', '2015-06-10')
summer = ee.Filter.date('2015-06-11', '2015-08-31')

# Create a composite image stacking the mean of each period (descending)
desc_change = ee.Image.cat(
    desc.filter(spring).mean(),
    desc.filter(late_spring).mean(),
    desc.filter(summer).mean(),
)

# Create a composite image stacking the mean of each period (ascending)
asc_change = ee.Image.cat(
    asc.filter(spring).mean(),
    asc.filter(late_spring).mean(),
    asc.filter(summer).mean(),
)

# Initialize the map
m = geemap.Map()
m.set_center(5.2013, 47.3277, 12)  # Center on a location (example: Dijon, France)

# Add the multi-temporal mean images to the map
m.add_layer(
    asc_change,
    {'min': -25, 'max': 5},
    'Multi-T Mean ASC',
    True
)
m.add_layer(
    desc_change,
    {'min': -25, 'max': 5},
    'Multi-T Mean DESC',
    True
)

# Display the map
m

Map(center=[47.3277, 5.2013], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

## Flood Detection Using Sentinel-1 SAR Imagery

In [4]:
# ===========================================
# Flood Detection Using Sentinel-1 SAR Imagery
# ===========================================

# Define the region of interest (ROI)
geometry = ee.Geometry.Polygon([
    [
        [106.34954329522984, -6.449380562588049],
        [107.33007308038609, -6.449380562588049],
        [107.33007308038609, -5.900522745264385],
        [106.34954329522984, -5.900522745264385]
    ]
])

# Initialize and center the map on the ROI
Map = geemap.Map()
Map.centerObject(geometry, 10)

# Sentinel-1 SAR image collection: before the flood event
sar_before = (
    ee.ImageCollection("COPERNICUS/S1_GRD")
    .filterDate('2019-12-20', '2019-12-29')
    .filterBounds(geometry)
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
    .select('VV')
    .map(lambda img: img.focalMean(60, 'square', 'meters')
         .copyProperties(img, img.propertyNames()))
)

# Sentinel-1 SAR image collection: after the flood event
sar_after = (
    ee.ImageCollection("COPERNICUS/S1_GRD")
    .filterDate('2019-12-30', '2020-01-03')
    .filterBounds(geometry)
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
    .select('VV')
    .map(lambda img: img.focalMean(60, 'square', 'meters')
         .copyProperties(img, img.propertyNames()))
)

# Compute the difference between before and after images
change = sar_before.min().subtract(sar_after.min())

# Load a permanent water mask from Dynamic World (2018–2021)
water_mask = (
    ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")
    .select('label')
    .filterDate('2018', '2021')
    .filterBounds(geometry)
    .mode()
    .eq(0)  # Class 0 = Water
    .Not()  # Invert: True where NOT permanent water
)

# Threshold change detection (>5 dB difference) and apply the water mask
thr = change.gt(5).updateMask(water_mask)
flooded = thr.updateMask(thr)

# Calculate flooded area in square kilometers
area_img = flooded.multiply(ee.Image.pixelArea().divide(1e6))
flood_area = area_img.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=geometry,
    scale=60
)

# Print the flooded area result
print(flood_area.getInfo())

# Add layers to the map
Map.addLayer(sar_before.min().clip(geometry), {}, "SAR Before")
Map.addLayer(sar_after.min().clip(geometry), {}, "SAR After")
Map.addLayer(change.clip(geometry), {}, "Change Detection")
Map.addLayer(flooded.clip(geometry), {"palette": ["blue"]}, "Detected Flooded Areas")

# Display the interactive map
Map

{'VV': 130.39439367585376}


Map(center=[-6.175128557541481, 106.839808187808], controls=(WidgetControl(options=['position', 'transparent_b…

## Search for SAR overlay in an area

### Oriniginal method

In [6]:
# Create a rectangular bounding box geometry
area = ee.Geometry.Rectangle([106.3, -6.4, 107.3, -5.9])

# Define the date range
start_date = '2019-12-30'
end_date = '2020-01-30'

# Load Sentinel-1 Image Collection for the given bounding box
sentinel1 = (ee.ImageCollection('COPERNICUS/S1_GRD')
             .filterBounds(area)
             .filterDate(start_date, end_date)
             .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
             .filter(ee.Filter.eq('instrumentMode', 'IW'))
             .select('VV'))

In [ ]:
image_list = sentinel1.toList(sentinel1.size())

for i in range(sentinel1.size().getInfo()): # 每次用 getInfo() 取 metadata 都要和 GEE server 請求一次
    image = ee.Image(image_list.get(i))
    segment_start_time = image.get('segmentStartTime')
    formatted_time = ee.Date(segment_start_time).format('YYYY-MM-dd HH:mm:ss').getInfo() if segment_start_time else "N/A" # 每次都要請求都要轉換一次時間
    print(f"Image {i+1} - segmentStartTime: {formatted_time}")

### Optimized version

In [12]:
from datetime import datetime

# Define the bounding box and date range
area = ee.Geometry.Rectangle([106.3, -6.4, 107.3, -5.9])
start_date = '2019-12-30'
end_date = '2020-01-30'

# Load Sentinel-1 Image Collection
sentinel1 = (ee.ImageCollection('COPERNICUS/S1_GRD')
             .filterBounds(area)
             .filterDate(start_date, end_date)
             .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
             .filter(ee.Filter.eq('instrumentMode', 'IW'))
             .select('VV'))

# Get all segmentStartTime and IDs in one go .aggregate_array()
times = sentinel1.aggregate_array('segmentStartTime').getInfo()
ids = sentinel1.aggregate_array('system:id').getInfo()

# Print results
for i, (img_id, timestamp) in enumerate(zip(ids, times)):
    if timestamp:
        # Always treat as milliseconds since epoch
        dt = datetime.utcfromtimestamp(timestamp / 1000.0)
        formatted_time = dt.strftime('%Y-%m-%d %H:%M:%S')
    else:
        formatted_time = "N/A"
    print(f"Image {i+1} - ID: {img_id} - segmentStartTime: {formatted_time}")

Image 1 - ID: COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20200102T111446_20200102T111515_030620_038227_6964 - segmentStartTime: 2020-01-02 11:14:47
Image 2 - ID: COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20200102T111515_20200102T111540_030620_038227_CF59 - segmentStartTime: 2020-01-02 11:14:47
Image 3 - ID: COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20200105T222535_20200105T222600_030671_0383EF_C8A2 - segmentStartTime: 2020-01-05 22:23:01
Image 4 - ID: COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20200114T111446_20200114T111515_030795_038847_558E - segmentStartTime: 2020-01-14 11:14:46
Image 5 - ID: COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20200114T111515_20200114T111540_030795_038847_0748 - segmentStartTime: 2020-01-14 11:14:46
Image 6 - ID: COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20200117T222535_20200117T222600_030846_038A11_8A56 - segmentStartTime: 2020-01-17 22:23:01
Image 7 - ID: COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20200122T223325_20200122T223350_030919_038C93_93B3 - segmentStartTime: 2020-01-22 22:30:52
Image 8 - ID: COPERN

## Flood event database

In [10]:
import pandas as pd

event_list = pd.read_csv("/home/ycchen/HAZAMA/FloodArchive.csv")

print(event_list.head())

   ID GlideNumber      Country OtherCountry       long       lat       Area  \
0   1           0      Algeria            0    5.23026  35.81420   92615.67   
1   2           0       Brazil            0  -45.34890 -18.71110  678498.82   
2   3           0  Phillipines            0  122.97400  10.02070   12846.03   
3   4           0    Indonesia            0  124.60600   1.01489   16542.12   
4   5           0   Mozambique            0   32.34910 -25.86930   20082.21   

       Began      Ended Validation  Dead  Displaced        MainCause  Severity  
0   1985/1/1   1985/1/5       News    26       3000       Heavy rain       1.0  
1  1985/1/15   1985/2/2       News   229      80000       Heavy rain       2.0  
2  1985/1/20  1985/1/21       News    43        444  Torrential rain       1.0  
3   1985/2/4  1985/2/18       News    21        300  Torrential rain       1.0  
4   1985/2/9  1985/2/11       News    19          0       Heavy rain       2.0  


In [ ]:
# get event time and location in event_list

def get_event_info(event_list):
    """
    Extracts event time and location from the event list DataFrame.
    
    Args:
        event_list (pd.DataFrame): DataFrame containing flood events with 'event_time' and 'location' columns.
    
    Returns:
        list: List of tuples containing (event_time, location).
    """
    events = []
    for index, row in event_list.iterrows():
        event_time = row['event_time']
        location = row['location']
        events.append((event_time, location))
    return events

In [4]:
Map = geemap.Map()

gfd = ee.ImageCollection('GLOBAL_FLOOD_DB/MODIS_EVENTS/V1')

hurricaneIsaacDartmouthId = 3977;
hurricaneIsaacUsa = ee.Image(
    gfd.filterMetadata('id', 'equals', hurricaneIsaacDartmouthId).first())

Map.setOptions('SATELLITE');
Map.setCenter(-90.2922, 29.4064, 9);
Map.addLayer(
  hurricaneIsaacUsa.select('flooded').selfMask(),
  {'min': 0, 'max': 1, 'palette': '001133'},
  'Hurricane Isaac - Inundation Extent')

Map

Map(center=[29.4064, -90.2922], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

In [13]:
def extract_event_details_for_gee(csv_filepath, bbox_offset=0.05):
    """
    Extracts time, latitude, and longitude for each event in the FloodArchive CSV
    and formats it for potential GEE queries.

    Args:
        csv_filepath (str): Path to the FloodArchive CSV file.
        bbox_offset (float): Half the width/height of the bounding box to create
                             around the event's lat/lon point (in degrees).
                             E.g., 0.05 creates a 0.1 x 0.1 degree box.
    """
    df = pd.read_csv(csv_filepath)
    print(f"Successfully loaded '{csv_filepath}'. Processing {len(df)} events.\n")

    # Ensure 'Began' and 'Ended' are treated as strings initially for direct formatting
    # then convert to datetime to validate and reformat, handling errors.
    df['Began_str'] = df['Began'].astype(str)
    df['Ended_str'] = df['Ended'].astype(str)

    output_lines = []

    for index, row in df.iterrows():
        try:
            lat = float(row['lat'])
            lon = float(row['long'])

            # Create a bounding box
            min_lon = lon - bbox_offset
            min_lat = lat - bbox_offset
            max_lon = lon + bbox_offset
            max_lat = lat + bbox_offset

            # Format dates, handling potential errors if dates are not standard
            try:
                start_date = pd.to_datetime(row['Began_str']).strftime('%Y-%m-%d')
            except ValueError:
                start_date = "INVALID_DATE" # Or however you want to handle unparseable dates
            
            try:
                end_date = pd.to_datetime(row['Ended_str']).strftime('%Y-%m-%d')
            except ValueError:
                end_date = "INVALID_DATE"

            event_id = row.get('ID', f"Event_{index+1}") # Use ID if available

            output_lines.append(f"# Event ID: {event_id}")
            output_lines.append(f"area = ee.Geometry.Rectangle([{min_lon:.4f}, {min_lat:.4f}, {max_lon:.4f}, {max_lat:.4f}])")
            output_lines.append(f"start_date = '{start_date}'")
            output_lines.append(f"end_date = '{end_date}'")
            output_lines.append("---") # Separator

        except ValueError:
            event_id = row.get('ID', f"Event_{index+1}")
            output_lines.append(f"# Event ID: {event_id} - SKIPPED (Invalid lat/lon)")
            output_lines.append("---")
            continue
        except Exception as e:
            event_id = row.get('ID', f"Event_{index+1}")
            output_lines.append(f"# Event ID: {event_id} - SKIPPED (Error: {e})")
            output_lines.append("---")
            continue
            
    # Print all collected lines
    # You can redirect this to a file if needed
    for line in output_lines:
        print(line)
        
    # If you want to save to a file:
    # output_filename = "gee_formatted_events.txt"
    # with open(output_filename, 'w') as f:
    #     for line in output_lines:
    #         f.write(line + "\n")
    # print(f"\nFormatted event details saved to {output_filename}")


# --- Configuration ---
csv_file = '/home/ycchen/HAZAMA/FloodArchive.csv'
# Define how large the bounding box around the event's point should be.
# 0.05 means the box will be 0.1 degrees wide and 0.1 degrees high.
# (approx. 11km x 11km at the equator). Adjust as needed.
bounding_box_half_size = 0.05 

# --- Run the extraction ---
extract_event_details_for_gee(csv_file, bbox_offset=bounding_box_half_size)
print("\nExtraction complete.")
print(f"Note: A bounding box of {2*bounding_box_half_size:.2f}x{2*bounding_box_half_size:.2f} degrees was created around each event's lat/lon.")
print("The 'Area' column from the CSV was not used to define these bounding boxes.")

Successfully loaded '/home/ycchen/HAZAMA/FloodArchive.csv'. Processing 5130 events.

# Event ID: 1
area = ee.Geometry.Rectangle([5.1803, 35.7642, 5.2803, 35.8642])
start_date = '1985-01-01'
end_date = '1985-01-05'
---
# Event ID: 2
area = ee.Geometry.Rectangle([-45.3989, -18.7611, -45.2989, -18.6611])
start_date = '1985-01-15'
end_date = '1985-02-02'
---
# Event ID: 3
area = ee.Geometry.Rectangle([122.9240, 9.9707, 123.0240, 10.0707])
start_date = '1985-01-20'
end_date = '1985-01-21'
---
# Event ID: 4
area = ee.Geometry.Rectangle([124.5560, 0.9649, 124.6560, 1.0649])
start_date = '1985-02-04'
end_date = '1985-02-18'
---
# Event ID: 5
area = ee.Geometry.Rectangle([32.2991, -25.9193, 32.3991, -25.8193])
start_date = '1985-02-09'
end_date = '1985-02-11'
---
# Event ID: 6
area = ee.Geometry.Rectangle([43.3100, -11.7016, 43.4100, -11.6016])
start_date = '1985-02-16'
end_date = '1985-02-28'
---
# Event ID: 7
area = ee.Geometry.Rectangle([175.6840, -37.2805, 175.7840, -37.1805])
start_date = 

In [ ]:
import pandas as pd
import ee
from datetime import datetime, timedelta

ee.Initialize()

# 設定參數
csv_filepath = '/home/ycchen/HAZAMA/FloodArchive.csv'
bbox_offset = 0.10  # 每邊 0.05 度，總共 0.1 度的方框

# 讀取 CSV
df = pd.read_csv(csv_filepath)
print(f"Loaded {len(df)} events from '{csv_filepath}'.")

# 將日期轉為字串備用
df['Began_str'] = df['Began'].astype(str)
df['Ended_str'] = df['Ended'].astype(str)

# 開始處理每一筆事件
for index, row in df.iterrows():   
    try:
        lat = float(row['lat'])
        lon = float(row['long'])
        min_lon = lon - bbox_offset
        min_lat = lat - bbox_offset
        max_lon = lon + bbox_offset
        max_lat = lat + bbox_offset
        area = ee.Geometry.Rectangle([min_lon, min_lat, max_lon, max_lat])

        # 轉換開始/結束日期
        try:
            start_dt = pd.to_datetime(row['Began_str']) - timedelta(days=30)
            start_date = pd.to_datetime(row['Began_str']).strftime('%Y-%m-%d')
        except:
            start_date = "INVALID_DATE"
        try:
            end_dt = pd.to_datetime(row['Ended_str']) + timedelta(days=30)
            end_date = pd.to_datetime(row['Ended_str']).strftime('%Y-%m-%d')
        except:
            end_date = "INVALID_DATE"

        event_id = row.get('ID', f"Event_{index+1}")
        print(f"\n# Event ID: {event_id}")
        if "INVALID_DATE" in [start_date, end_date]:
            print("Skipped (Invalid date)")
            continue

        # 查詢 Sentinel-1 影像
        sentinel1 = (ee.ImageCollection('COPERNICUS/S1_GRD')
                     .filterBounds(area)
                     .filterDate(start_date, end_date)
                     .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
                     .filter(ee.Filter.eq('instrumentMode', 'IW'))
                     .select('VV'))

        times = sentinel1.aggregate_array('segmentStartTime').getInfo()
        ids = sentinel1.aggregate_array('system:id').getInfo()

        if not ids:
            print("No matching images.")
            continue

        for i, (img_id, timestamp) in enumerate(zip(ids, times)):
            if timestamp:
                dt = datetime.utcfromtimestamp(timestamp / 1000.0)
                formatted_time = dt.strftime('%Y-%m-%d %H:%M:%S')
            else:
                formatted_time = "N/A"
            print(f"Image {i+1} - ID: {img_id} - Time: {formatted_time}")
    except Exception as e:
        event_id = row.get('ID', f"Event_{index+1}")
        print(f"\n# Event ID: {event_id} - SKIPPED (Error: {e})")
        continue

Loaded 5130 events from '/home/ycchen/HAZAMA/FloodArchive.csv'.

# Event ID: 1
No matching images.

# Event ID: 2
No matching images.

# Event ID: 3
No matching images.

# Event ID: 4
No matching images.

# Event ID: 5
No matching images.

# Event ID: 6
No matching images.

# Event ID: 7
No matching images.

# Event ID: 8
No matching images.

# Event ID: 9
No matching images.

# Event ID: 10
No matching images.

# Event ID: 11
No matching images.

# Event ID: 12
No matching images.

# Event ID: 13
No matching images.

# Event ID: 14
No matching images.

# Event ID: 15
No matching images.

# Event ID: 16
No matching images.

# Event ID: 17
No matching images.

# Event ID: 18
No matching images.

# Event ID: 19
No matching images.

# Event ID: 20
No matching images.

# Event ID: 21
No matching images.

# Event ID: 22
No matching images.

# Event ID: 23
No matching images.

# Event ID: 24
No matching images.

# Event ID: 25
No matching images.

# Event ID: 26
No matching images.

# Event 

In [17]:
import pandas as pd
import ee
import json
from datetime import datetime

ee.Initialize()

# === 設定參數 ===
csv_filepath = '/home/ycchen/HAZAMA/FloodArchive.csv'
bbox_offset = 0.05  # 每邊擴張 0.05 度
output_json_path = 'matched_events.json'

# === 讀取事件資料 ===
df = pd.read_csv(csv_filepath)
df['Began_str'] = df['Began'].astype(str)
df['Ended_str'] = df['Ended'].astype(str)
print(f"Loaded {len(df)} events from '{csv_filepath}'.")

# === 儲存所有事件的影像查詢結果 ===
event_matches = []

# === 處理每筆事件 ===
for index, row in df.iterrows():
    try:
        lat = float(row['lat'])
        lon = float(row['long'])
        min_lon = lon - bbox_offset
        min_lat = lat - bbox_offset
        max_lon = lon + bbox_offset
        max_lat = lat + bbox_offset
        area = ee.Geometry.Rectangle([min_lon, min_lat, max_lon, max_lat])

        try:
            start_date = pd.to_datetime(row['Began_str']).strftime('%Y-%m-%d')
        except:
            start_date = "INVALID_DATE"
        try:
            end_date = pd.to_datetime(row['Ended_str']).strftime('%Y-%m-%d')
        except:
            end_date = "INVALID_DATE"

        event_id = row.get('ID', f"Event_{index+1}")

        if "INVALID_DATE" in [start_date, end_date]:
            print(f"# Event ID: {event_id} - Skipped (Invalid date)")
            continue

        sentinel1 = (ee.ImageCollection('COPERNICUS/S1_GRD')
                     .filterBounds(area)
                     .filterDate(start_date, end_date)
                     .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
                     .filter(ee.Filter.eq('instrumentMode', 'IW'))
                     .select('VV'))

        times = sentinel1.aggregate_array('segmentStartTime').getInfo()
        ids = sentinel1.aggregate_array('system:id').getInfo()

        matched_images = []
        for img_id, timestamp in zip(ids, times):
            dt = datetime.utcfromtimestamp(timestamp / 1000.0) if timestamp else None
            formatted_time = dt.strftime('%Y-%m-%d %H:%M:%S') if dt else "N/A"
            matched_images.append({
                'Image_ID': img_id,
                'Image_Time': formatted_time
            })

        event_matches.append({
            'Event_ID': event_id,
            'Lat': lat,
            'Lon': lon,
            'Start_Date': start_date,
            'End_Date': end_date,
            'Bounding_Box': [min_lon, min_lat, max_lon, max_lat],
            'Images': matched_images
        })

        print(f"# Event ID: {event_id} - Found {len(matched_images)} images")

    except Exception as e:
        print(f"# Event ID: {row.get('ID', f'Event_{index+1}')} - Skipped (Error: {e})")
        continue

# === 寫入 JSON ===
with open(output_json_path, 'w') as f:
    json.dump(event_matches, f, indent=2)

print(f"\nDone. Results saved to: {output_json_path}")

Loaded 5130 events from '/home/ycchen/HAZAMA/FloodArchive.csv'.
# Event ID: 1 - Found 0 images
# Event ID: 2 - Found 0 images
# Event ID: 3 - Found 0 images
# Event ID: 4 - Found 0 images
# Event ID: 5 - Found 0 images
# Event ID: 6 - Found 0 images
# Event ID: 7 - Found 0 images
# Event ID: 8 - Found 0 images
# Event ID: 9 - Found 0 images
# Event ID: 10 - Found 0 images
# Event ID: 11 - Found 0 images
# Event ID: 12 - Found 0 images
# Event ID: 13 - Found 0 images
# Event ID: 14 - Found 0 images
# Event ID: 15 - Found 0 images
# Event ID: 16 - Found 0 images
# Event ID: 17 - Found 0 images
# Event ID: 18 - Found 0 images
# Event ID: 19 - Found 0 images
# Event ID: 20 - Found 0 images
# Event ID: 21 - Found 0 images
# Event ID: 22 - Found 0 images
# Event ID: 23 - Found 0 images
# Event ID: 24 - Found 0 images
# Event ID: 25 - Found 0 images
# Event ID: 26 - Found 0 images
# Event ID: 27 - Found 0 images
# Event ID: 28 - Found 0 images
# Event ID: 29 - Found 0 images
# Event ID: 30 - 